In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yadneshmundhada","key":"b8b6f5ac1e548b716da83b811e5cee1c"}'}

In [3]:
import tensorflow as tf
tf.__version__

!sudo pip3 install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
device_list = tf.test.gpu_device_name()
device_list

'/device:GPU:0'

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -f train.zip.001 -p /content -c diabetic-retinopathy-detection
!7z x train.zip.001.zip
!rm train.zip.001.zip
!7z x train.zip.001
!rm train.zip.001

100% 7.81G/7.81G [03:16<00:00, 47.3MB/s]
100% 7.81G/7.81G [03:16<00:00, 42.7MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 8391159087 bytes (8003 MiB)

Extracting archive: train.zip.001.zip
--
Path = train.zip.001.zip
Type = zip
Physical Size = 8391159087
64-bit = +

  0%      0% - train.zip.001                      1% - train.zip.001                      2% - train.zip.001                      3% - train.zip.001                      4% - train.zip.001                      5% - train.zip.001                      

In [ ]:
! kaggle competitions download -f train.zip.002 -p /content -c diabetic-retinopathy-detection
!7z x train.zip.002.zip
!rm train.zip.002.zip
!7z x train.zip.002
!rm train.zip.002

In [ ]:
! kaggle competitions download -f train.zip.003 -p /content -c diabetic-retinopathy-detection
!7z x train.zip.003.zip
!rm train.zip.003.zip
!7z x train.zip.003
!rm train.zip.003

In [ ]:
! kaggle competitions download -f train.zip.004 -p /content -c diabetic-retinopathy-detection
!7z x train.zip.004.zip
!rm train.zip.004.zip
!7z x train.zip.004
!rm train.zip.004

In [ ]:
! kaggle competitions download -f train.zip.005 -p /content -c diabetic-retinopathy-detection
!7z x train.zip.005.zip
!rm train.zip.005.zip
!7z x train.zip.005
!rm train.zip.005

In [7]:
! kaggle competitions download -f trainLabels.csv.zip -p /content -c diabetic-retinopathy-detection
! unzip /content/trainLabels.csv.zip
! rm trainLabels.csv.zip

  0% 0.00/69.4k [00:00<?, ?B/s]
100% 69.4k/69.4k [00:00<00:00, 54.5MB/s]
Archive:  /content/trainLabels.csv.zip
  inflating: trainLabels.csv         


In [8]:
import pandas as pd
retina_df = pd.read_csv('/content/trainLabels.csv');
retina_df

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
...,...,...
35121,44347_right,0
35122,44348_left,0
35123,44348_right,0
35124,44349_left,0


In [9]:
import shutil
import sys
import os
train_dir=r'../content/train'
DR=r'../content/train/'
# if not os.path.exists(DR):
#   os.mkdir(DR)
for filename,class_name in retina_df.values:
  if(not os.path.exists(DR+str(class_name))):
    os.mkdir(DR+str(class_name))
  src = train_dir+'/'+filename+'.jpeg'
  dst = DR + str(class_name)+'/'+filename+'.jpeg'
  try:
    shutil.move(src,dst);
    print("done"+filename);
  except IOError as e:
    print('Unable to move')

Streaming output truncated to the last 5000 lines.
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unable to move
Unab

In [10]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np
import pandas as pd

from subprocess import check_output
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
batch_size = 32
nb_classes = 5
nb_epoch = 5
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [11]:
train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(200, 200), 
        batch_size=16,
        class_mode='categorical',
        subset='training')  

validation_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(200, 200),
        batch_size=16,
        class_mode='categorical',
        subset='validation')

Found 6728 images belonging to 5 classes.
Found 1680 images belonging to 5 classes.


In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (200, 200, 3),
                                include_top = False,
                                weights = 'imagenet')

87924736/87910968 [==============================] - 0s 0us/step


In [13]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [14]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc') is not None and logs.get('acc')>0.959):
      self.model.stop_training = True

In [15]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

x = layers.Flatten()(pre_trained_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (5, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 99, 99, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 99, 99, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [19]:
batch_size=16
callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 25,
            validation_steps = 50,
            callbacks=[callbacks])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/25
100/100 [==============================] - 291s 3s/step - loss: 1.1900 - accuracy: 0.6702 - val_loss: 0.9821 - val_accuracy: 0.7113
Epoch 2/25
100/100 [==============================] - 286s 3s/step - loss: 0.9935 - accuracy: 0.6994 - val_loss: 0.9493 - val_accuracy: 0.7412
Epoch 3/25
100/100 [==============================] - 284s 3s/step - loss: 0.8936 - accuracy: 0.7431 - val_loss: 0.9491 - val_accuracy: 0.7375
Epoch 4/25
100/100 [==============================] - 280s 3s/step - loss: 0.9674 - accuracy: 0.7129 - val_loss: 0.8335 - val_accuracy: 0.7487
Epoch 5/25
100/100 [==============================] - 287s 3s/step - loss: 0.9231 - accuracy: 0.7250 - val_loss: 0.8718 - val_accuracy: 0.7375
Epoch 6/25
100/100 [==============================] - 288s 3s/step - loss: 0.9322 - accuracy: 0.7156 - val_loss: 0.8635 - val_accuracy: 0.7487
Epoch 7/25
100/100 [==============================] - 282s 3s/step - loss: 0.8591 - accuracy: 0.7425 - val_loss: 0.9654 - val_accuracy: 0.7287

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
batch_size=16
checkpoint = ModelCheckpoint("save.h5", monitor='val_accuracy', 
                             verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=1, mode='auto')
mod = model.fit_generator(steps_per_epoch=train_generator.samples // batch_size,
                           generator=train_generator, 
                           validation_data= validation_generator, 
                           validation_steps=validation_generator.samples // batch_size,
                           epochs=25,callbacks=[checkpoint,early])